# Predicción de severidad de accidentes de tránsito



1.   Tenemos un archivo con datos: train.csv el cual contiene información sobre accidentes de tránsito en EEUU y un archivo test4alumnxs.csv que tiene la misma info pero sin las etiquetas para Severity (más información abajo).
2.   El objetivo es usar los datos para construir un modelo que prediga la severidad del accidente (columna 'Severity', donde 0 es baja severidad y 1 es alta severidad). Para esto, primero puedo usar los datos del archivo train.csv para ajustar los parámetros del modelo, ajustar los hiperparámetros, combinar los features para agregar nuevos, estandarizar los datos, etc. Al terminar esto, tendré un modelo entrenado en estos datos, y una idea de que tan bien funciona. Luego generaremos un vector de probabilidades sobre el test4alumnos.csv para tratar de predecir la severidad del accidente.

# Los datos
Los datos corresponden a registros de accidentes de tránsito en EEUU entre 2016 y 2021.

Las primeras dos columnas ("unnamed") no sirve nara nada y se pueden descartar. 

La columna 'Severity' contiene la severidad del accidente (0 significa baja, 1 significa alta)

Luego tenemos la hora de comienzo y final del accidente, la latitud/longitud inicial y final del accidente (entre que valores queda delimitado), la distancia de camino afectada por el accidente, una descripción verbal hecha por un humano, número y nombre de la calle donde ocurrió, lado de la calle (izquierda o derecha), ciudad, condado, estado, código postal, huso horario, código de aeropuerto donde se encuentra la estación meterológica más cercana al accidente, hora en la cual se midió el clima,temperatura, temperatura del viento, humedad, presión, visibilidad, dirección del viento, condición climática (despejado, sol, nieve, etc), presencia de comodidades ("amenities") en la cercanía del accidente, presencia de loma de burro, presencia de un cruce, presencia de señal de ceder paso, presencia de unión de calles, presencia de cartel de no salida, presencia de vías de tren, presencia de rotonda, presencia de estación, presencia de signo de parar, presencia de moderadores de tráfico, presencia de señales de tránsito, presencia de calle en forma de U ("loop"), día u oscuridad de acuerdo a la salida del sol, día u oscuridad de acuerdo a la penumbra civil (si es o no necesario utilizar alumbramiento eléctrico), día u oscuridad de acuerdo a la penumbra náutica, día u oscuridad de acuerdo a la penumbra astronómica.

No todos estos campos son necesariamente útiles, y tampoco no todos están en un formato numérico directamente utilizable (por ejemplo, hay strings en la parte de descripción del accidente). Será parte del trabajo decidir que features quedarse, que features sumar, y como extraer información relevante de los features más complejos. 

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

drive.mount('/content/drive') # montamos el drive

# cargamos el dataframe de entrenamiento y el dataframe de testeo
# el dataframe de entrenamiento tiene toda la información, incluyendo la columna 'Severity' (severidad)
# esta es la columna que vamos a entrenar el modelo para predecir: 0=severidad baja; 1=severidad alta.
# en cambio, el dataframe de test no tiene esta columna: son los datos que usamos para generar la predicción que se entrega 
filename_train = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/train.csv' 
filename_test = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/test4alumnxs.csv' 

df_train = pd.read_csv(filename_train)
df_test = pd.read_csv(filename_test)
df_test.columns

In [ ]:
df_train.columns

In [ ]:
df_train["Severity"].value_counts()

In [ ]:
# Como el dataset no parece tener un desbalance importante, sigo adelante.

In [ ]:
df_train["Weather_Condition"].value_counts()

In [ ]:
# Agrego algunas columnas a train.csv y a test4alumnxs.csv para testear mis hipótesis:
# 1) Los accidentes más severos son los que se producen en autopistas o carreteras de alta velocidad (por ejemplo, rutas nacionales).
# 2) Los accidentes más severos contienen una advertencia del tipo "blocked", "multi" o "closed road" (u otras)
# 3) Los accidentes más severos se producen de noche.
# 4) Los accidentes más severos se producen en condiciones climatológicas desfavorables (lluvia, nieve, niebla, etc).
def highway_or_road(a):
    if "highway" in a.lower():
        return True
    elif "hwy" in a.lower():
        return True
    elif "road" in a.lower():
      return True
    elif "rd" in a.lower():
      return True
    elif "route" in a.lower():
      return True
    else:
        return False
df_train["Highway_or_road"] = df_train["Street"].apply(lambda x: highway_or_road(x))
df_test["Highway_or_road"] = df_test["Street"].apply(lambda x: highway_or_road(x))

def serious(a):
    #if "drive with caution" in a.lower():
        #return True
    if "closed" in a.lower():
        return True
    elif "blocked" in a.lower():
      return True
    elif "closure" in a.lower():
      return True
    elif "multi" in a.lower():
      return True
    elif "serious" in a.lower():
      return True
    else:
        return False
df_train["Serious"] = df_train["Description"].apply(lambda x: serious(x))
df_test["Serious"] = df_test["Description"].apply(lambda x: serious(x))

def night(a):
  if "Night" in a.lower():
    return True
  else:
    return False

df_train["Night"] = df_train["Civil_Twilight"].apply(lambda x: night(x))
df_test["Night"] = df_test["Civil_Twilight"].apply(lambda x: night(x))

def adverse_weather(a):
  if "rain" in a.lower():
    return True
  elif "snow" in a.lower():
    return True
  elif "fog" in a.lower():
    return True
  elif "smoke" in a.lower():
    return True
  else:
    return False

df_train["Adverse_Weather"] = df_train["Weather_Condition"].apply(lambda x: adverse_weather(x))
df_test["Adverse_Weather"] = df_test["Weather_Condition"].apply(lambda x: adverse_weather(x))

df_train.columns

In [ ]:
df_test.columns

In [ ]:
df_test.shape

In [ ]:
df_train

Luego, sacamos el vector de etiquetas (severidad).

En este modelo vamos a usar el subconjunto de features que corresponde a variables climáticas: temperatura, temperatura del viento, humedad, presión, visibilidad, velocidad del viento y precipitación así como otras correspondientes al contexto del accidente (si era un cruce de calles, si ocurrió en autopista, si el clima era adverso, etc.).

Con esto armamos la matriz X. Luego, separamos un 70%/30% para train y test del modelo.



In [ ]:
y = np.array(df_train['Severity']) # armo el vector de etiquetas

# vamos a entrenar un modelo con estas variables
columnas = ['Temperature(F)', 'Wind_Chill(F)', 'Distance(mi)', 'Stop', 'Humidity(%)','Junction','Crossing', "Highway_or_road", "Serious", "Night", "Adverse_Weather",'Traffic_Signal', 'Traffic_Calming','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Precipitation(in)']

# extraigo la matriz de features X
X = np.array(df_train[columnas])

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)


In [ ]:
# pruebo con un random forest así como viene de fábrica
clf_rf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                             max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, 
                             random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)  

# ajusto el modelo
clf_rf.fit(X_train, y_train) #Como veo que es el mejor modelo, lo nombro de diferente manera así puedo luego armar el vector de etiquetas.

# armo el vector de predicciones.
y_hat_test = clf_rf.predict_proba(X_test)[:, 1]
y_hat_train = clf_rf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_train, roc_test

In [ ]:
auc_values = [] # aca es donde van a ir a parar los AUCs de cada fold
scores = np.array([])     # aca es donde van a ir a parar los scores computados para todos los casos
indices = np.array([])    # aca es donde van a ir a parar los indices correspondientes a las entradas de scores
for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True).split(X, y): # va generando los indices que corresponden a train y test en cada fold
    X_train, X_test = X[train_index], X[test_index] # arma que es dato de entrenamiento y qué es dato de evaluación
    y_train, y_test = y[train_index], y[test_index]     # idem con los targets

    clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                             max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, 
                             random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None) # Inicializamos nuevamente el modelo. max_iter es la cantidad de iteraciones maximas del algoritmo de optimizacion de parametros antes de detenerse.
    clf.fit(X_train, y_train) # Ajustamos el modelo con los datos de entrenamiento

    probas_test = clf.predict_proba(X_test)  # probabilidades con datos de evaluación
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, probas_test[:,1]) # para tener curva ROC con datos de entrenamiento
    auc_test = roc_auc_score(y_test, probas_test[:,1]) #  AUC con datos de evaluación

    auc_values.append(auc_test)
    scores = np.concatenate((scores,probas_test[:,1]),axis=0)
    indices = np.concatenate((indices,test_index),axis=0)

print("Estos son los valores AUC para cada fold:")
print(auc_values)
print("Estos es el promedio de todos los AUC:")
print(np.mean(auc_values))
print("Estos son las probabilidades para cada sample:")
print(scores)
print("Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):")
print(indices)

In [ ]:
# pruebo con una regresion logistica así como viene de fábrica
clf = LogisticRegression()  

# ajusto el modelo
clf.fit(X_train, y_train)

# armo el vector de predicciones.
y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_train, roc_test

In [ ]:
auc_values = [] # aca es donde van a ir a parar los AUCs de cada fold
scores = np.array([])     # aca es donde van a ir a parar los scores computados para todos los casos
indices = np.array([])    # aca es donde van a ir a parar los indices correspondientes a las entradas de scores
for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True).split(X, y): # va generando los indices que corresponden a train y test en cada fold
    X_train, X_test = X[train_index], X[test_index] # arma que es dato de entrenamiento y qué es dato de evaluación
    y_train, y_test = y[train_index], y[test_index]     # idem con los targets

    clf = clf # Inicializamos nuevamente el modelo. max_iter es la cantidad de iteraciones maximas del algoritmo de optimizacion de parametros antes de detenerse.
    clf.fit(X_train, y_train) # Ajustamos el modelo con los datos de entrenamiento

    probas_test = clf.predict_proba(X_test)  # probabilidades con datos de evaluación
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, probas_test[:,1]) # para tener curva ROC con datos de entrenamiento
    auc_test = roc_auc_score(y_test, probas_test[:,1]) #  AUC con datos de evaluación

    auc_values.append(auc_test)
    scores = np.concatenate((scores,probas_test[:,1]),axis=0)
    indices = np.concatenate((indices,test_index),axis=0)

print("Estos son los valores AUC para cada fold:")
print(auc_values)
print("Estos es el promedio de todos los AUC:")
print(np.mean(auc_values))
print("Estos son las probabilidades para cada sample:")
print(scores)
print("Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):")
print(indices)

In [ ]:
# pruebo con un arbol de decision así como viene de fábrica
clf = DecisionTreeClassifier()  

# ajusto el modelo
clf.fit(X_train, y_train)

# armo el vector de predicciones.
y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_train, roc_test

In [ ]:
auc_values = [] # aca es donde van a ir a parar los AUCs de cada fold
scores = np.array([])     # aca es donde van a ir a parar los scores computados para todos los casos
indices = np.array([])    # aca es donde van a ir a parar los indices correspondientes a las entradas de scores
for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True).split(X, y): # va generando los indices que corresponden a train y test en cada fold
    X_train, X_test = X[train_index], X[test_index] # arma que es dato de entrenamiento y qué es dato de evaluación
    y_train, y_test = y[train_index], y[test_index]     # idem con los targets

    clf = clf # Inicializamos nuevamente el modelo. max_iter es la cantidad de iteraciones maximas del algoritmo de optimizacion de parametros antes de detenerse.
    clf.fit(X_train, y_train) # Ajustamos el modelo con los datos de entrenamiento

    probas_test = clf.predict_proba(X_test)  # probabilidades con datos de evaluación
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, probas_test[:,1]) # para tener curva ROC con datos de entrenamiento
    auc_test = roc_auc_score(y_test, probas_test[:,1]) #  AUC con datos de evaluación

    auc_values.append(auc_test)
    scores = np.concatenate((scores,probas_test[:,1]),axis=0)
    indices = np.concatenate((indices,test_index),axis=0)

print("Estos son los valores AUC para cada fold:")
print(auc_values)
print("Estos es el promedio de todos los AUC:")
print(np.mean(auc_values))
print("Estos son las probabilidades para cada sample:")
print(scores)
print("Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):")
print(indices)

In [ ]:
# pruebo con una red de perceptrones multilayer así como viene de fábrica
clf = MLPClassifier()  

# ajusto el modelo
clf.fit(X_train, y_train)

# armo el vector de predicciones.
y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_train, roc_test

In [ ]:
auc_values = [] # aca es donde van a ir a parar los AUCs de cada fold
scores = np.array([])     # aca es donde van a ir a parar los scores computados para todos los casos
indices = np.array([])    # aca es donde van a ir a parar los indices correspondientes a las entradas de scores
for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True).split(X, y): # va generando los indices que corresponden a train y test en cada fold
    X_train, X_test = X[train_index], X[test_index] # arma que es dato de entrenamiento y qué es dato de evaluación
    y_train, y_test = y[train_index], y[test_index]     # idem con los targets

    clf = clf # Inicializamos nuevamente el modelo. max_iter es la cantidad de iteraciones maximas del algoritmo de optimizacion de parametros antes de detenerse.
    clf.fit(X_train, y_train) # Ajustamos el modelo con los datos de entrenamiento

    probas_test = clf.predict_proba(X_test)  # probabilidades con datos de evaluación
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, probas_test[:,1]) # para tener curva ROC con datos de entrenamiento
    auc_test = roc_auc_score(y_test, probas_test[:,1]) #  AUC con datos de evaluación

    auc_values.append(auc_test)
    scores = np.concatenate((scores,probas_test[:,1]),axis=0)
    indices = np.concatenate((indices,test_index),axis=0)

print("Estos son los valores AUC para cada fold:")
print(auc_values)
print("Estos es el promedio de todos los AUC:")
print(np.mean(auc_values))
print("Estos son las probabilidades para cada sample:")
print(scores)
print("Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):")
print(indices)

Por último, veo que el RandomForest es quien obtiene los mejores resultados. Aplicamos el modelo ya entrenado a los datos de test4alumnxs.csv y guardo el vector "predicciones" y lo descargo, tal como muestra el screenshot de más abajo.

In [ ]:
X = np.array(df_test[columnas]) # cargo el dataset de testeo

predicciones = clf_rf.predict_proba(X)[:, 1] # obtengo el vector de probabilidades
predicciones.size
# Controlo que el vector de probabilidades tenga el mismo largo que las filas de df_test


In [ ]:
print(predicciones)

# lo guardo en el espacio de trabajo de colab
np.savetxt('predicciones.csv', predicciones, delimiter=",")